In [1]:
import pyodbc 
import pandas as pd
# Some other example server values are
# server = 'myserver,port' # to specify an alternate port
server = 'localhost' 
database = 'Vaccination'
username = 'vaccine' 
password = 'vaccine' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 
	Sep 24 2019 13:48:23 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 22000: )



In [2]:
date = pd.read_csv(r'C:\Users\htran\OneDrive\Documents\Project\py\date.csv',index_col=False, delimiter = ',')
country = pd.read_csv(r'C:\Users\htran\OneDrive\Documents\Project\py\country.csv',index_col=False, delimiter = ',')
vaccine = pd.read_csv(r'C:\Users\htran\OneDrive\Documents\Project\py\vaccine.csv',index_col=False, delimiter = ',')
fact = pd.read_csv(r'C:\Users\htran\OneDrive\Documents\Project\csv\country_vaccinations.csv',index_col=False,low_memory=False, delimiter = ',')

In [3]:
dfD = pd.DataFrame(date)
dfC = pd.DataFrame(country)
dfV = pd.DataFrame(vaccine)


In [5]:
#removes duplicates
dfD = dfD.drop_duplicates()
dfC = dfC.drop_duplicates()
dfV = dfV.drop_duplicates()

In [8]:
for row in dfC.itertuples():
    cursor.execute('''
                INSERT INTO Dimension.Country (iso_code, country)
                VALUES (?,?)
                ''',
                row.iso_code, 
                row.country
                )
cnxn.commit()

In [9]:
for row in dfD.itertuples():
    cursor.execute('''
                INSERT INTO Dimension.Date (datekey, fulldate)
                VALUES (?,?)
                ''',
                row.datekey, 
                row.fulldate
                )
cnxn.commit()

In [51]:
fact = pd.read_csv(r'C:\Users\htran\OneDrive\Documents\Project\csv\country_vaccinations.csv',index_col=False,low_memory=False, delimiter = ',')
dfF = pd.DataFrame(fact)

In [52]:
#df.drop(['daily_vaccinations_raw', 'total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred','daily_vaccinations_per_million'], axis=1)
#drop empty rows from dataset
dfF.drop(dfF.index[86512:1048575]) 

,VaccinationID,iso_code,datekey,vaccinekey,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations
0,1.0,AFG,20210222.0,7,0.0,0.0,NaN,NaN
1,2.0,AFG,20210223.0,7,NaN,NaN,NaN,1367.0
2,3.0,AFG,20210224.0,7,NaN,NaN,NaN,1367.0
3,4.0,AFG,20210225.0,7,NaN,NaN,NaN,1367.0
4,5.0,AFG,20210226.0,7,NaN,NaN,NaN,1367.0
...,...,...,...,...,...,...,...,...
86507,86508.0,ZWE,20220325.0,2,8691642.0,4814582.0,3473523.0,69579.0
86508,86509.0,ZWE,20220326.0,2,8791728.0,4886242.0,3487962.0,83429.0
86509,86510.0,ZWE,20220327.0,2,8845039.0,4918147.0,3493763.0,90629.0
86510,86511.0,ZWE,20220328.0,2,8934360.0,4975433.0,3501493.0,100614.0


In [45]:
for row in dfF.itertuples():
    cursor.execute('''
                INSERT INTO Fact.Vaccinations (VaccinationID,iso_code,datekey,vaccinekey,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.VaccinationID,   
                row.iso_code, 
                row.datekey,
                row.vaccinekey,
                row.total_vaccinations,
                row.people_vaccinated,
                row.people_fully_vaccinated,
                row.daily_vaccinations,
                )
cnxn.commit()

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 10 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')